# #0 Journal paper critique

# #1 Reproduce figure with $\textrm{Bokeh}$

# #2 Reproduce figure in $\textrm{matplotlib}$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

load data

In [ ]:
data_dir = 'hw_2_data/'

In [ ]:
ny_temp = np.loadtxt(data_dir + 'ny_temps.txt', skiprows=1)
google_stock = np.loadtxt(data_dir + 'google_data.txt', skiprows=1)
yahoo_stock = np.loadtxt(data_dir + 'yahoo_data.txt', skiprows=1)

In [ ]:
google_t, google_v = google_stock[:, 0], google_stock[:, 1]
yahoo_t, yahoo_v = yahoo_stock[:, 0], yahoo_stock[:, 1]
ny_t, ny_v = ny_temp[:, 0], ny_temp[:, 1]

In [ ]:
from matplotlib.ticker import MultipleLocator

In [ ]:
fs=16
fig, ax1 = plt.subplots(figsize=[8,6])

lns1 = ax1.plot(yahoo_t, yahoo_v, 'purple', label='Yahoo! Stock Value')
lns2 = ax1.plot(google_t, google_v, 'b-', label='Google Stock Value')
ax1.set_xlabel('Date (MJD)', fontsize=fs)
ax1.set_ylabel('Value (Dollars)', fontsize=fs)

ax1.set_ylim([-20, 780])
ax1.set_xlim([49000, 55000])

# add minor ticks
ax1.xaxis.set_minor_locator(MultipleLocator(200))
ax1.yaxis.set_minor_locator(MultipleLocator(20))

# set font for title
font = {'family': 'sans-serif','color':  'black', 
        'weight': 'bold','size': fs}
ax1.set_title('New York Temperature, Google and Yahoo!', fontdict=font)

# turn off major and minor ticks from upper x axis
ax1.tick_params(axis='x', which='both', top='off')        

ax2 = ax1.twinx()
lns3 = ax2.plot(ny_t, ny_v, 'r--', label='NY Mon. High Temp')
ax2.set_ylabel('Temperature $(^\circ \mathrm{F})$', fontsize=fs)
ax2.set_ylim([-150, 100])
ax2.yaxis.set_minor_locator(MultipleLocator(10))

# add legend
lns = lns1+lns2+lns3
labs = [l.get_label() for l in lns]
ax1.legend(lns, labs, loc='center left', frameon=False)

plt.show()

# #3 Generic "Brushing" code

In [1]:
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib inline

Read dataset with many rows and multiple columns (variables/parameters).

In [2]:
data_dir = 'hw_2_data/'
filename = data_dir + 'flowers.csv'

In [3]:
!tail -n 5 hw_2_data/flowers.csv

6.7,3.0,5.2,2.3,virginica
6.3,2.5,5.0,1.9,virginica
6.5,3.0,5.2,2.0,virginica
6.2,3.4,5.4,2.3,virginica
5.9,3.0,5.1,1.8,virginica


In [4]:
# datatype
dt = [('sepalLength', 'f4'), ('sepalWidth', 'f4'),
     ('petalLength', 'f4'), ('petalWidth', 'f4'), 
      ('species', 'S10')]

names = ['sepalLength', 'sepalWidth', 'petalLength', 'petalWidth', 'species']
formats = ['f4', 'f4', 'f4', 'f4', 'S10']
dt = {'names': names, 'formats':formats}
# dataset
ds = np.loadtxt(filename, delimiter=',', skiprows=1, dtype=dt)

Make subplots

In [5]:
setosa_ind = ds['species'] == b'setosa'
versicolor_ind = ds['species'] == b'versicolor'
virginica_ind = ds['species'] == b'virginica'

In [6]:
for i, var_i in enumerate(names):
    print(i, var_i)

0 sepalLength
1 sepalWidth
2 petalLength
3 petalWidth
4 species


In [7]:
# ds['sepalLength'][setosa_ind]

Create subplot 

In [8]:
# import ipywidgets as widgets
# %matplotlib notebook

In [9]:
# fig, axes = plt.subplots(4, 4, sharey='row', sharex='col', figsize=[8,8])
# for i, var_i in enumerate(names[:4]):
#     for j, var_j in enumerate(names[:4]):
#         data_i = ds[var_i]
#         data_j = ds[var_j]
#         ax = axes[i, j]
#         ax.plot(data_j, data_i, 'o', markersize=1)
#         if i == j:
#             ax.text(0.1, 0.8, var_i, transform=ax.transAxes)
            
# w = widgets.HTML()

# def onclick(event):
#     w.value = 'button=%d, x=%d, y=%d, xdata=%f, ydata=%f'%(
#               event.button, event.x, event.y, event.xdata, event.ydata)

# cid = fig.canvas.mpl_connect('button_release_event', onclick)
# w            

In [35]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

class Brushing(object):
    def __init__(self, fig):
        self.rect = None
        self.fig = fig
        self.axes = fig.axes
        self.x0 = None
        self.y0 = None
        self.x1 = None
        self.y1 = None
        
        self.ax = None
        self.press = None
        
        self.fig.canvas.mpl_connect('button_press_event', self.on_press)
        self.fig.canvas.mpl_connect('button_release_event', self.on_release)
        self.fig.canvas.mpl_connect('motion_notify_event', self.on_motion)


    def on_press(self, event):
        print('press')
        if not self.rect:
            self.rect = Rectangle((0,0), 0, 0, facecolor='grey', alpha = 0.2)
            self.ax = event.inaxes
            self.ax.add_patch(self.rect)
        if self.ax != event.inaxes:
            self.ax = event.inaxes
            self.rect.set_visible(False)
            del self.rect
            self.rect = Rectangle((0,0), 0, 0, facecolor='grey', alpha = 0.2)
            self.ax.add_patch(self.rect)
        else:
            self.rect.set_width(0)
            self.rect.set_height(0)
        self.press = True
        self.x0 = event.xdata
        self.y0 = event.ydata

    def on_release(self, event):
        print('release')
        self.press = None
        if event.inaxes != self.rect.axes: return
        self.x1 = event.xdata
        self.y1 = event.ydata
        self.rect.set_width(self.x1 - self.x0)
        self.rect.set_height(self.y1 - self.y0)
        self.rect.set_xy((self.x0, self.y0))        
        self.ax.figure.canvas.draw()

    def on_motion(self, event):
        if self.press is None: return
        if event.inaxes != self.rect.axes: return
        
        x1 = event.xdata
        y1 = event.ydata
        self.rect.set_width(x1 - self.x0)
        self.rect.set_height(y1 - self.y0)
        self.rect.set_xy((self.x0, self.y0))
        self.ax.figure.canvas.draw()
            

In [36]:
fig, axes = plt.subplots(4, 4, sharey='row', sharex='col', figsize=[8,8])
for i, var_i in enumerate(names[:4]):
    for j, var_j in enumerate(names[:4]):
        data_i = ds[var_i]
        data_j = ds[var_j]
        ax = axes[i, j]
        ax.plot(data_j, data_i, 'o', markersize=2)
        if i == j:
            ax.text(0.1, 0.8, var_i, transform=ax.transAxes)
            
a = Brushing(fig)
# w = widgets.HTML()
plt.show()


press
release
press
release
press
release
press
release
press
release


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

class Annotate(object):
    def __init__(self):
        self.ax = plt.gca()
        self.rect = Rectangle((0,0), 0, 0)
        
        self.x0 = None
        self.y0 = None
        self.x1 = None
        self.y1 = None
        
        self.press = None
        self.ax.add_patch(self.rect)
        
        self.ax.figure.canvas.mpl_connect('button_press_event', self.on_press)
        self.ax.figure.canvas.mpl_connect('button_release_event', self.on_release)
        self.ax.figure.canvas.mpl_connect('motion_notify_event', self.on_motion)


    def on_press(self, event):
        print('press')
        self.press = True
        self.x0 = event.xdata
        self.y0 = event.ydata

    def on_release(self, event):
        print('release')
        self.press = None
        if event.inaxes != self.rect.axes: return
        self.x1 = event.xdata
        self.y1 = event.ydata
        self.rect.set_width(self.x1 - self.x0)
        self.rect.set_height(self.y1 - self.y0)
        self.rect.set_xy((self.x0, self.y0))        
        self.ax.figure.canvas.draw()

    def on_motion(self, event):
        if self.press is None: return
        if event.inaxes != self.rect.axes: return
        
        self.x1 = event.xdata
        self.y1 = event.ydata
        self.rect.set_width(self.x1 - self.x0)
        self.rect.set_height(self.y1 - self.y0)
        self.rect.set_xy((self.x0, self.y0))
        self.ax.figure.canvas.draw()

        
a = Annotate()
plt.show()

In [ ]:
"""
Illustrate the figure and axes enter and leave events by changing the
frame colors on enter and leave
"""
import matplotlib.pyplot as plt

def enter_axes(event):
    print('enter_axes', event.inaxes)
    event.inaxes.patch.set_facecolor('yellow')
    event.canvas.draw()

def leave_axes(event):
    print('leave_axes', event.inaxes)
    event.inaxes.patch.set_facecolor('white')
    event.canvas.draw()

def enter_figure(event):
    print('enter_figure', event.canvas.figure)
    event.canvas.figure.patch.set_facecolor('red')
    event.canvas.draw()

def leave_figure(event):
    print('leave_figure', event.canvas.figure)
    event.canvas.figure.patch.set_facecolor('grey')
    event.canvas.draw()

# fig1 = plt.figure()
# fig1.suptitle('mouse hover over figure or axes to trigger events')
# ax1 = fig1.add_subplot(211)
# ax2 = fig1.add_subplot(212)

# fig1.canvas.mpl_connect('figure_enter_event', enter_figure)
# fig1.canvas.mpl_connect('figure_leave_event', leave_figure)
# fig1.canvas.mpl_connect('axes_enter_event', enter_axes)
# fig1.canvas.mpl_connect('axes_leave_event', leave_axes)

fig2 = plt.figure()
fig2.suptitle('mouse hover over figure or axes to trigger events')
ax1 = fig2.add_subplot(211)
ax2 = fig2.add_subplot(212)

fig2.canvas.mpl_connect('figure_enter_event', enter_figure)
fig2.canvas.mpl_connect('figure_leave_event', leave_figure)
fig2.canvas.mpl_connect('axes_enter_event', enter_axes)
fig2.canvas.mpl_connect('axes_leave_event', leave_axes)

plt.show()